In [1]:
import pandas as pd
from GameFeatures import GameFeatures
from PreGameFeatures import PreGameFeatures
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('gamesDB.csv')

In [3]:
df.columns

Index(['Unnamed: 0', 'data', 'time_casa', 'time_visitante', 'resultado',
       'id_partida', 'link'],
      dtype='object')

In [4]:
df.drop(columns='Unnamed: 0', inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560 entries, 0 to 4559
Data columns (total 6 columns):
data              4560 non-null object
time_casa         4560 non-null object
time_visitante    4560 non-null object
resultado         4560 non-null object
id_partida        4560 non-null object
link              4560 non-null object
dtypes: object(6)
memory usage: 213.8+ KB


In [6]:
df.data[0]

'2007-05-12 19:10:00'

In [7]:
df.data = pd.to_datetime(df.data)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560 entries, 0 to 4559
Data columns (total 6 columns):
data              4560 non-null datetime64[ns]
time_casa         4560 non-null object
time_visitante    4560 non-null object
resultado         4560 non-null object
id_partida        4560 non-null object
link              4560 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 213.8+ KB


In [9]:
mask = df.data.dt.year > 2009

In [10]:
df[mask].head()

,data,time_casa,time_visitante,resultado,id_partida,link
1140,2010-05-08 19:30:00,Botafogo,Santos,3-3,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...
1141,2010-05-08 19:30:00,Atlético GO,Grêmio,0-0,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...
1142,2010-05-08 19:35:00,Palmeiras,Vitória,1-0,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...
1143,2010-05-09 17:00:00,Flamengo,São Paulo,1-1,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...
1144,2010-05-09 17:00:00,Atlético MG,Vasco,2-1,gsm_id_919738,https://www.academiadasapostasbrasil.com/stats...


In [11]:
links = df.link[1140:1150]

In [12]:
home_team = []
away_team = []
home_team_score = []
away_team_score = []
referee = []
stadium = []
city = []
datetime = []
game = []
eventList1st = []
eventList2nd = []
lineupList = []
sublineupList = []
statsHome = []
statsAway = []

In [13]:
import concurrent.futures

In [14]:
#from GameFeatures import GameFeatures
#from random_ua import get_random_ua, get_random_agent
#print(get_random_agent())

#url = 'https://www.academiadasapostasbrasil.com/stats/match/brasil-stats/brasileirao-serie-a/vitoria/ponte-preta/2218001/1/live'
#print(GameFeatures(url))


In [15]:
#%load_ext cython

In [ ]:


i = 0
games_len = len(links)
print(f'games_len: ${games_len}')
sfx='prelive'

with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(GameFeatures, url): url for url in links}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            #data = future.result()
            home_teamT, away_teamT, home_team_scoreT, away_team_scoreT, refereeT, stadiumT, cityT, datetimeT, gameT, eventList1stT, eventList2ndT, lineupListT, sublineupListT, statsHomeT, statsAwayT = future.result()
            
            pre_game=PreGameFeatures(url[:-4]+sfx)
            pre_game.build()
            
            home_team.append(home_teamT) 
            away_team.append(away_teamT) 
            home_team_score.append(home_team_scoreT) 
            away_team_score.append(away_team_scoreT) 
            referee.append(refereeT) 
            stadium.append(stadiumT) 
            city.append(cityT) 
            datetime.append(datetimeT) 
            game.append(gameT) 
            eventList1st.append(eventList1stT) 
            eventList2nd.append(eventList2ndT) 
            lineupList.append(lineupListT) 
            sublineupList.append(sublineupListT) 
            statsHome.append(statsHomeT) 
            statsAway.append(statsAwayT)
            i+=1
            #print(f'home_teamT:{home_teamT}')
            print('Game number: '+str(i)+'/'+str(games_len))
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        #else:
            #print('%r page ' % (url))

games_len: $10
Game number: 1/10
Game number: 2/10
Game number: 3/10
Game number: 4/10


In [ ]:
df2 = pd.DataFrame({'home_team':home_team, 
                    'away_team':away_team, 
                    'home_team_score':home_team_score, 
                    'away_team_score':away_team_score, 
                    'referee':referee, 
                    'stadium':stadium, 
                    'city':city, 
                    'datetime':datetime, 
                    'game':game, 
                    'eventList1st':eventList1st, 
                    'eventList2nd':eventList2nd, 
                    'lineupList':lineupList, 
                    'sublineupList':sublineupList, 
                    'statsHome':statsHome, 
                    'statsAway':statsAway})


In [ ]:
df2.to_csv('soccerDBNew2.csv')